# 프로젝트 : 얼굴을 인식하여 캐릭터 씌우기
- face detection : 사람 얼굴의 특징을 이용해 찾는방법
- face recognition : 누구의 얼굴인지를 찾는 방법 

In [13]:
import cv2
import mediapipe as mp

# 얼굴을 찾고 , 찾은 얼굴에 표시를 해주기 위한 변수 정의 
mp_face_detection = mp.solutions.face_detection  # 얼굴 검출을 위해 face-detection 모듈을 사용 
mp_drawing = mp.solutions.drawing_utils         # 얼굴의 특징을 그리기 위한 drawing_utils 모듈 사용 

# 동영상 파일 열기 
cap = cv2.VideoCapture("face_video.mp4")

# 이미지 불러오기 
img_right_eye = cv2.imread("right_eye.png")
img_left_eye = cv2.imread("left_eye.png")
img_nose = cv2.imread("nose.png")

# model_selection = 0(2미터 이내 근거리 영상), 1(5미터 이내 중거리 영상)
# min_detection_confidence = 얼굴로 보는 임계값 (얼굴로 인식하는 신뢰도)
                                    
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while cap.isOpened():
      success, image = cap.read()
      if not success:
        break

      image.flags.writeable = False
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      results = face_detection.process(image)

    # Draw the face detection annotations on the image.
      image.flags.writeable = True
      image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
      if results.detections:
          # 6개 특징 : 오/왼눈, 코끝, 입 중심, 오/왼귀 ( 귀 구슬점, 이주 )
        for detection in results.detections:
           # mp_drawing.draw_detection(image, detection)
           # print(detection)
          
            # 특정 위치 가져오기 
            keypoints = detection.location_data.relative_keypoints
            right_eye = keypoints[0]
            left_eye = keypoints[1]
            nose_tip = keypoints[2]
        

            h, w, _ = image.shape # height,width, channel  : 이미지로부터 세로, 가로 크기 가져옴
            right_eye = (int(right_eye.x * w)-20,int(right_eye.y*h)-100) # 이미지 내에서 실제 좌표 (x,y)
            left_eye = (int(left_eye.x * w)+20,int(left_eye.y*h)-100)
            nose_tip = (int(nose_tip.x * w),int(nose_tip.y*h))

        
             # 양 눈에 동그라미 그리기 
         #   cv2.circle(image, right_eye,radius=50, color=(255,0,0), thickness=10, lineType=cv2.LINE_AA)
         #   cv2.circle(image, left_eye,radius=50, color=(0,255,0), thickness=10, lineType=cv2.LINE_AA)  
         #   cv2.circle(image, nose_tip,radius=75, color=(0,255,255), thickness=10, lineType=cv2.LINE_AA)  

             # 각 특징에 이미지 넣기 
            image[right_eye[1]-50:right_eye[1]+50, right_eye[0]-50:right_eye[0]+50] = img_right_eye
            image[left_eye[1]-50:left_eye[1]+50, left_eye[0]-50:left_eye[0]+50] = img_left_eye
            image[nose_tip[1]-50:nose_tip[1]+50, nose_tip[0]-150:nose_tip[0]+150] = img_nose
              
          
    # Flip the image horizontally for a selfie-view display.
      cv2.imshow('MediaPipe Face Detection', cv2.resize(image,None,fx=0.5,fy=0.5))
      
      if cv2.waitKey(5)== ord("q"):
        break
cap.release()
cv2.destroyAllWindows()